# Working out database loading

As this is now, it will fill in the complete database.
If you really want to reset things, you should delete the SQLite db file first.

This'll do for now.

In [1]:
from spotclient import Client
import models

db = models.Database()
session = models.get_session(create_all=True)

c = Client()
ab = c.allbirds()
latest = ab[0]

In [2]:
playlist = db.insert_playlist_from_json(session, latest)
# track_ids = [t.spotify_id for t in playlist.tracks]
# spot_tracks = dict((t['id'], t) for t in c.tracks(track_ids))
# for track in playlist.tracks:
#     t = c._sp.track(playlist.tracks[0].spotify_id)
#     t.album = models.Album.get_or_create(session, )

    
    
session.commit()


In [17]:
tracks = c.playlist_tracks(latest['id'])

In [18]:
tracks[0]

{'artist': 'Tortoise',
 'title': 'Cornpone Brunch',
 'album': 'Tortoise',
 'url': 'https://open.spotify.com/track/6I5Jew45g2z0s5UayE0rHw'}